# APC-CLI 基本チュートリアル 実行フロー

各ステップを上から順に実行することで、AutoPrivacy DCR上で関数を実行することができます。
AutoPrivacy DCRの操作をCLIから行うツールとして、APC-CLIというコマンドラインツールを用います。

**このJupyterNotebookは [dcr-docs-examplesリポジトリ](https://github.com/acompany-develop/dcr-docs-examples) に含まれています。リポジトリをクローンし、ステップ3の環境変数を設定すれば、説明に従って各セルを実行するだけでAutoPrivacy DCR上で処理を実行できます。**

## 概要

このスクリプトは以下の主要なステップで構成されています：
1. チュートリアル全体概要
2. AutoPrivacy DCR 概要
3. 環境変数設定
4. APC-CLI のセットアップ
5. プロファイル設定
6. 入出力の設定ファイルの生成
7. 認証とヘルスチェック
8. プロジェクト設定
9. 関数の準備
10. 関数ストレージへのアップロード
11. Cleanroom デプロイ
12. データの Cleanroom へのコピー
13. Cleanroom の実行
14. 結果のダウンロード
15. クリーンアップ

## 1. チュートリアル全体概要

このチュートリアルでは、AutoPrivacy DCRを使用してデータの安全な共有と処理を実現する方法を説明します。具体的には、二者間でデータを共有し、Cleanroom上でjoin関数を実行することで、プライバシーを保護しながらデータ処理を行います。join関数の全実装は[こちらのコード](https://github.com/acompany-develop/dcr-docs-examples/blob/main/functions/join/function/handler.py)のようになります。

**前提条件：**
- 設定情報（CLIENT_ID、CLIENT_SECRET等）
- Python 3.10
- OS: macOS、Linux
- アーキテクチャ: x86_64、ARM64

## 2. AutoPrivacy DCR 概要
Cleanroomを用いることで、二者間で安全にデータを共有し実行することが可能となります。APC-CLI実行時にprofile（user）を指定することで、異なるユーザーとして実行することができます。profileによってユーザーが区別されるため、異なるマシンから実行することが可能です（同一のマシンから実行することも可能）。
具体的なフロー図は以下のようになります。

<details><summary>フロー図</summary>
<img src="../img/apc_sequence.svg" width="800">
</details>

## 3. 環境変数設定

実行に必要な環境変数を設定します。`API_URL`, `ATTESTATION_URL`, `ATTESTATION_API_VERSION`, `MR_ENCLAVE`, `MR_SIGNER`, `CLIENT_ID1`, `CLIENT_SECRET1`, `CLIENT_ID2`, `CLIENT_SECRET2`, `PROJECT_ID`という環境変数の値は、AutoPrivacy DCRサービス提供者から提供されるconfig.tomlファイルに記載されています。ただし、ダブルクオーテーションはつけずに値だけを環境変数に代入してください。

なお、データ共有を行う二者間では、`CLIENT_ID`と`CLIENT_SECRET`だけ異なり、それ以外の環境変数の値（`API_URL`, `ATTESTATION_URL`, `ATTESTATION_API_VERSION`, `MR_ENCLAVE`, `MR_SIGNER`, `PROJECT_ID`）は同一になります。

In [ ]:
# マシンに保存される設定に紐づく識別子。ユーザーが自由に設定することが出来る。
%env profile1=user1
# マシンに保存される設定に紐づく識別子。ユーザーが自由に設定することが出来る。
%env profile2=user2

%env API_URL=
%env ATTESTATION_URL=
%env ATTESTATION_API_VERSION=
%env MR_ENCLAVE=
%env MR_SIGNER=
%env CLIENT_ID1=
%env CLIENT_SECRET1=
%env CLIENT_ID2=
%env CLIENT_SECRET2=
%env PROJECT_ID=

# 入出力設定ファイルのパス
%env ENCRYPTED_FILES_PATH=encrypted-files.yaml

## 4. APC-CLI のセットアップ

詳細なAPC-CLIのセットアップ方法については、[こちら](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/getting-started/installation.html)から参照できます。

まず、`apc_cli_version`は[こちら](https://github.com/acompany-develop/apc-cli/releases)を参照し、APC-CLIのバージョンを指定してください。以下は例として、`3.0.1`を指定しています。

In [ ]:
%env APC_CLI_VERSION=3.0.1

指定したバージョンでAPC-CLIをダウンロードします。

**注意**: このステップではwgetを使用します。wgetがインストールされていない場合は、以下のコマンドでインストールしてください：
- **macOS**: `brew install wget`
- **Linux (Ubuntu/Debian)**: `sudo apt-get install wget`

In [ ]:
%%bash

# OSを判定
OS=$(uname -s)

case $OS in
    "Darwin")
        # macOS
        echo "macOS detected"
        wget https://github.com/acompany-develop/apc-cli/releases/download/$APC_CLI_VERSION/apc-darwin-arm64.zip
        unzip apc-darwin-arm64.zip
        mv apc-darwin-arm64 apc
        chmod +x apc
        rm apc-darwin-arm64.zip
        ;;
    "Linux")
        # Linux
        echo "Linux detected"
        wget https://github.com/acompany-develop/apc-cli/releases/download/$APC_CLI_VERSION/apc-linux-x64.zip
        unzip apc-linux-x64.zip
        mv apc-linux-x64 apc
        chmod +x apc
        rm apc-linux-x64.zip
        ;;
    *)
        echo "Unknown OS: $OS"
        exit 1
        ;;
esac

ダウンロードしたAPC-CLIのパスを通します。

In [ ]:
import os

os.environ["PATH"] = os.path.dirname(os.path.abspath("apc")) + ":" + os.environ["PATH"]

APC-CLIが正しくインストールされているか確認します。

In [ ]:
%%bash
apc --version

## 5. プロファイル設定
各プロファイルに対して設定情報を入力し、User IDを生成します。
今回のチュートリアルでは標準出力されるUserIDを環境変数に読み込むために以下のようにコマンドを実行します。
コマンドの詳細は [configure コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/configure.html) から参照できます。

まずは、`profile1`について設定します。出力された`USER_ID`は次の入出力設定のステップで使用します。

In [ ]:
%%bash
expect <<EOF
spawn apc configure --profile $profile1
expect "API URL: "
send "$API_URL\r"
expect "Attestation URL: "
send "$ATTESTATION_URL\r"
expect "Attestation API Version: "
send "$ATTESTATION_API_VERSION\r"
expect "Attestation MR Enclave: "
send "$MR_ENCLAVE\r"
expect "Attestation MR Signer: "
send "$MR_SIGNER\r"
expect "Client ID: "
send "$CLIENT_ID1\r"
expect "Client Secret: "
send "$CLIENT_SECRET1\r"
expect eof
EOF

次に、`profile2`について設定します。こちらも同様に、出力された`USER_ID`は次の入出力設定のステップで使用します。

In [ ]:
%%bash
expect <<EOF
spawn apc configure --profile $profile2
expect "API URL: "
send "$API_URL\r"
expect "Attestation URL: "
send "$ATTESTATION_URL\r"
expect "Attestation API Version: "
send "$ATTESTATION_API_VERSION\r"
expect "Attestation MR Enclave: "
send "$MR_ENCLAVE\r"
expect "Attestation MR Signer: "
send "$MR_SIGNER\r"
expect "Client ID: "
send "$CLIENT_ID2\r"
expect "Client Secret: "
send "$CLIENT_SECRET2\r"
expect eof
EOF

## 6. 入出力の設定ファイルの生成

入出力の設定ファイルを`$ENCRYPTED_FILES_PATH`に作成します。この設定ファイルでは、入力データと出力データにアクセス可能なUser IDを指定します。
詳細は[こちら](https://acompany-develop.github.io/autoprivacy-cloud/apc-dcr/user-guide/user-files/definition-files.html)から参照できます。

**注意: 以下のコマンド実行前に**`<USER_1_ID>`**と**`<USER_2_ID>`**を5で出力された**`profile1`**と**`profile2`**のUser IDに置き換えてください。**

In [ ]:
%%bash
cat > $ENCRYPTED_FILES_PATH << EOF
inputs:
  input_1: &user_1_id <USER_1_ID>
  input_2: &user_2_id <USER_2_ID>
outputs:
  output_1: *user_1_id
  output_2: *user_2_id
EOF
cat $ENCRYPTED_FILES_PATH

## 7. 認証とヘルスチェック

各プロファイルでログインします。コマンドの詳細は [auth-login コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/auth-login.html) から参照できます。

In [ ]:
%%bash
apc auth-login --profile $profile1
apc auth-login --profile $profile2

各プロファイルで、サーバーに対してヘルスチェックを行い、正常にサーバーと通信ができているかを確認します。コマンドの詳細は [healthcheck コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/healthcheck.html) から参照できます。

In [ ]:
%%bash
apc healthcheck --profile $profile1
apc healthcheck --profile $profile2

## 8. プロジェクト設定

各プロファイルで同じプロジェクトを設定します。コマンドの詳細は [set-project コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/set-project.html) から参照できます。

In [ ]:
%%bash
apc set-project $PROJECT_ID --profile $profile1
apc set-project $PROJECT_ID --profile $profile2

## 9. 関数の準備

Cleanroom上で実行する関数のパスを設定します。今回のチュートリアルでは、リポジトリの中のjoin関数のパスを`FUNCTION_SOURCE_PATH`に指定します。

ファイル・ディレクトリ構成としては、今回は以下のようなものを想定します。ディレクトリ構成の詳細については[ドキュメント](https://acompany-develop.github.io/autoprivacy-cloud/apc-dcr/user-guide/user-files/function-directory.html)から参照できます。

```
FUNCTION_SOURCE_PATH
├── function # 関数のパス
│   └── handler.py
│   └── packages # 依存パッケージのパス
├── inputs # 入力データのパス
│   ├── input_1
│   └── input_2
└── outputs # 出力データのパス
    ├── output_1
    └── output_2
```

なお、[このディレクトリ](https://github.com/acompany-develop/dcr-docs-examples/tree/main/functions)の中の関数はCleanroom上で実行可能であり、これらの関数のパスを指定することも可能です。

In [ ]:
import os

%env FUNCTION_SOURCE_PATH=../../functions/join

FUNCTION_SOURCE_PATH = os.environ.get("FUNCTION_SOURCE_PATH")

%env FUNCTION_SOURCE_PATH={FUNCTION_SOURCE_PATH}
# 関数のパス
%env FUNCTION_DIRECTORY_PATH={FUNCTION_SOURCE_PATH}/function
# 入力データのパス
%env INPUT_1_PATH={FUNCTION_SOURCE_PATH}/inputs/input_1
%env INPUT_2_PATH={FUNCTION_SOURCE_PATH}/inputs/input_2
# 出力データのパス
%env OUTPUT_1_PATH={FUNCTION_SOURCE_PATH}/outputs/output_1
%env OUTPUT_2_PATH={FUNCTION_SOURCE_PATH}/outputs/output_2

Cleanroom上での関数実行で使用するライブラリを`packages`ディレクトリにインストールします。

In [ ]:
%%bash
pip install --platform manylinux2014_x86_64 \
    --only-binary=:all: \
    --python-version 3.10  \
    --target=$FUNCTION_DIRECTORY_PATH/packages \
    -r $FUNCTION_SOURCE_PATH/requirements.txt

## 10. 関数ストレージへのアップロード

関数ディレクトリのパスを指定した上で、実行する関数をCleanroom上にアップロードします。コマンドの詳細は [function-storage コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/function-storage.html#function-storage-upload) から参照できます。なお、出力された`FunctionStoragePath`は次のCleanroomデプロイのステップで使用します。

In [ ]:
%%bash
apc function-storage upload --source $FUNCTION_DIRECTORY_PATH --profile $profile1

## 11. Cleanroom デプロイ

アップロードされた関数を使用してCleanroomアプリケーションをデプロイします。
コマンドの詳細は [cleanroom deploy コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom.html#cleanroom-deploy) から参照できます。

**注意: 以下のコマンド実行前に**`--source`**オプションの引数の<FUNCTION_STORAGE_PATH>を10で出力された**`FunctionStoragePath`**の値に置き換えてください。**

In [ ]:
%%bash
apc cleanroom deploy \
    --source <FUNCTION_STORAGE_PATH> \
    --name join_app \
    --runtime python3.10 \
    --handler handler.run \
    --encrypted-files $ENCRYPTED_FILES_PATH \
    --memory 2 \
    --profile $profile1

## 12. データの Cleanroom へのコピー

各プロファイルから、ローカルの入力データをCleanroom上にコピーします。コマンドの詳細は [cleanroom copyコマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom-data.html) から参照できます。

In [ ]:
%%bash
apc cleanroom data cp $INPUT_1_PATH join_app:input_1 --profile $profile1
apc cleanroom data cp $INPUT_2_PATH join_app:input_2 --profile $profile2

## 13. Cleanroom の実行

デプロイされたCleanroom上のアプリケーションを実行します。コマンドの詳細は [cleanroom run コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom.html#cleanroom-run) から参照できます。

In [ ]:
%%bash
apc cleanroom run join_app --profile $profile1

## 14. 結果のダウンロード

実行結果を各プロファイルにダウンロードします。コマンドの詳細は [cleanroom data コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom-data.html) から参照できます。

In [ ]:
%%bash
apc cleanroom data cp join_app:output_1 $OUTPUT_1_PATH --profile $profile1
apc cleanroom data cp join_app:output_2 $OUTPUT_2_PATH --profile $profile2

## 15. クリーンアップ

DCR上にアップロードした関数ディレクトリやデプロイしたCleanroomアプリケーションを使用しない場合はクリーンアップすることをお勧めします。それぞれの手順は以下の通りです。

Cleanroom上のアプリケーションを削除します。コマンドの詳細は [cleanroom delete コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom.html#cleanroom-delete) から参照できます。

In [ ]:
%%bash
apc cleanroom delete join_app --profile $profile1

Cleanroom上にアップロードされた関数を削除します。コマンドの詳細は [function-storage delete コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/function-storage.html#function-storage-delete) から参照できます。

**注意: 以下のコマンド実行前に<FUNCTION_STORAGE_PATH>を10で出力された`FunctionStoragePath`の値に置き換えてください。**

In [ ]:
%%bash
apc function-storage delete <FUNCTION_STORAGE_PATH> --profile $profile1